In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")


# class MyQueryEndEventHandler(BaseEventHandler):
#     @classmethod
#     def class_name(cls) -> str:
#         """Class name."""
#         return "MyQueryEndEventHandler"

#     def handle(self, event) -> None:
#         """Logic for handling event."""
#         if isinstance(event, QueryEndEvent):
#             print(self.class_name())
#             with open("log.txt", "a") as f:
#                 f.write(str(event))
#                 f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # modify root dispatcher

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
dispatcher

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher = instrument.get_dispatcher("llama_index.core.base.query_engine")

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.query_engine', event_handlers=[], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x118542ec0>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher.root

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_result = query_engine.query("Who is Paul?")

RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent


In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='BaseRetriever.retrieve-85a56b6e-936e-4469-828d-182b4bcb759c', parent_id=None, start_time=datetime.datetime(2024, 3, 6, 11, 14, 54, 420305), end_time=datetime.datetime(2024, 3, 6, 11, 14, 54, 557290), duration=0.136985),
 SimpleSpan(id_='LLM.predict-ad3d8669-03fe-4eef-b43b-dbba0a7dd6f2', parent_id='Refine.get_response-43eb56e9-1de8-4506-ab35-f618241557c3', start_time=datetime.datetime(2024, 3, 6, 11, 14, 54, 561170), end_time=datetime.datetime(2024, 3, 6, 11, 14, 56, 71075), duration=1.509905),
 SimpleSpan(id_='Refine.get_response-43eb56e9-1de8-4506-ab35-f618241557c3', parent_id='CompactAndRefine.get_response-b7749ea7-084d-4b14-91ae-f55fb55eea88', start_time=datetime.datetime(2024, 3, 6, 11, 14, 54, 559704), end_time=datetime.datetime(2024, 3, 6, 11, 14, 56, 71339), duration=1.511635),
 SimpleSpan(id_='CompactAndRefine.get_response-b7749ea7-084d-4b14-91ae-f55fb55eea88', parent_id='BaseSynthesizer.synthesize-a20edc50-74a1-4454-b8de-09fd1aff6759', start_time=datetime.date

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent
QueryEndEvent


In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='BaseRetriever.retrieve-b7162490-df65-4889-b7ff-e30486aa2d47', parent_id='BaseQueryEngine.query-ec37c135-a604-431c-93b0-ac59e04b08f2', start_time=datetime.datetime(2024, 3, 5, 15, 8, 24, 783652), end_time=datetime.datetime(2024, 3, 5, 15, 8, 25, 94676), duration=0.311024),
 SimpleSpan(id_='LLM.predict-907308ff-eb95-4ab3-b501-e52acdc8cdb7', parent_id='Refine.get_response-fbf03458-b07c-4695-b819-18ddf9d04049', start_time=datetime.datetime(2024, 3, 5, 15, 8, 25, 103614), end_time=datetime.datetime(2024, 3, 5, 15, 8, 26, 630280), duration=1.526666),
 SimpleSpan(id_='Refine.get_response-fbf03458-b07c-4695-b819-18ddf9d04049', parent_id='CompactAndRefine.get_response-98c996cc-7fde-4b33-9626-97ab049e6354', start_time=datetime.datetime(2024, 3, 5, 15, 8, 25, 99861), end_time=datetime.datetime(2024, 3, 5, 15, 8, 26, 630540), duration=1.530679),
 SimpleSpan(id_='CompactAndRefine.get_response-98c996cc-7fde-4b33-9626-97ab049e6354', parent_id='BaseSynthesizer.synthesize-625a8f49-0d4e

### Very Primitive Tree Viz

In [ ]:
%pip install treelib -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from treelib import Node, Tree

tree = Tree()

In [ ]:
for span in reversed(dispatcher.span_handler.completed_spans[:6]):
    tree.create_node(
        tag=f"{span.id_} ({span.duration})",
        identifier=span.id_,
        parent=span.parent_id,
        data=span.duration,
    )

In [ ]:
print(tree.show(stdout=False))

BaseQueryEngine.query-ec37c135-a604-431c-93b0-ac59e04b08f2 (1.847953)
├── BaseRetriever.retrieve-b7162490-df65-4889-b7ff-e30486aa2d47 (0.311024)
└── BaseSynthesizer.synthesize-625a8f49-0d4e-4a99-8839-934a22b86df5 (1.535965)
    └── CompactAndRefine.get_response-98c996cc-7fde-4b33-9626-97ab049e6354 (1.535171)
        └── Refine.get_response-fbf03458-b07c-4695-b819-18ddf9d04049 (1.530679)
            └── LLM.predict-907308ff-eb95-4ab3-b501-e52acdc8cdb7 (1.526666)

